In [24]:
import pandas as pd
import numpy as np
import tushare as ts
import time
import sys

from datetime import timedelta
from pyecharts import options as opts
from pyecharts.charts import Kline, Line, Bar, Grid,Scatter

# Base1 Environment
Provide global variables

In [25]:
class Environment(object):
    """
    support other module with global variables
    """
    today = time.strftime("%Y%m%d", time.localtime())
    commision_fee = 3e-4
    init_value = 200_000
    E2C_dict = {
        'date': '日期',
        'code': '代码',
        'name': '简称',
        'direction': '买卖方向',
        'price_deal': '成交价格',
        'quantity': '成交数量',
        'turnover': '买卖金额',
        'fee': '佣金费',
        'tax': '印花税',
        'holding': '持仓',
        'WACP': '成本额',
        'price_cost': '成本',
        'price_current': '现价',
        'pct_today(%)': '今日涨幅(%)',
        'pct_cum(%)': '累计涨幅(%)',
        'value': '市值',
        'gain_today': '今日收益',
        'gain_cum': '累计收益',
        'capital': '初始资金',
        'asset': '总资产',
        'cash': '现金',
        'value_total': '总市值',
        'gain_total': '总盈亏',
        'return_total(%)': '总回报率(%)',
        'return_holding(%)': '持仓收益率(%)',
        'position_rate': '仓位',
    }
    C2E_dict = {c: e for e, c in E2C_dict.items()}

# Base2 Transaction
read the transaction book

In [26]:
class Transaction(Environment):
    '''
    read transactions book
    '''

    def __init__(self):
        self.transactions = self.read()

    def read(self):
        transactions = pd.read_excel('交易流水.xlsx')
        transactions.rename(columns=self.C2E_dict, inplace=True)
        transactions.quantity.where(
            transactions.direction == '买入', abs(transactions.quantity)*-1, inplace=True)
        transactions['turnover'] = transactions.price_deal * \
            transactions.quantity
        transactions['fee'] = np.maximum(
            5, abs(transactions.turnover)*self.commision_fee)
        transactions['tax'] = abs(
            transactions.loc[transactions.direction == '卖出', 'turnover']*1e-3)
        transactions['code'] = transactions['code'].str.upper()
        return transactions

# Base3 DataStored
Download specific stock price information

In [27]:
class DataStored(Transaction):
    '''
    Download the stock price data for the whole period

    '''

    def __init__(self, start=None, end=None):
        Transaction.__init__(self)
        self.start = start
        self.end = end
        self.data = self.pull_data()

    def pull_data(self):
        stocks = {}

        end = self.today if self.end == None else self.end
        start = self.transactions.date[0].strftime(
            "%Y%m%d") if self.start == None else self.start

        # date 00:00+ hours to ensure it cover the day
        transactions = self.transactions[self.transactions.date <= (
            pd.to_datetime(end)+timedelta(0.9))]

        # find the unique stocks
        code = transactions.code.drop_duplicates().tolist()

        for i in range(len(code)):
            sys.stdout.write(
                'Loading stocks {}, proceeding {}/{}\r'.format(code[i], i+1, len(code)))
            sys.stdout.flush()
            # identity stock or etf
            if code[i][0] == '6' or code[i][0] == '0':
                data = ts.pro_bar(
                    ts_code=code[i], adj='qfq', start_date=start, end_date=end)
            elif code[i][0] == '5':
                data = ts.pro_bar(
                    ts_code=code[i], asset='FD', start_date=start, end_date=end)
            else:
                print('can not find {}'.format(code[i]))
            data.trade_date = pd.to_datetime(data.trade_date)
            data.set_index('trade_date', inplace=True)

            stocks[code[i]] = data
            
        sys.stdout.write('\nData downloaded succesfully\n')
        sys.stdout.flush()
        data = pd.concat(stocks, sort=False).unstack(level=0)
        data.columns.names = ['Attributes', 'code']
        data.reset_index(inplace=True)
        data.rename(columns={'trade_date': 'date'}, inplace=True)
        return data

# Base4 DailyReport
Using transactions and provided date, check daily asset value

In [28]:
class DailyReport(Transaction):
    '''
    Input: Data:DataFrame, date:str
    reading transactions until current day
    quoting closing price of current day
    Output: DataFrame: postion value and portfolio value
    '''

    def __init__(self, data: 'DataFrame', date: str = None):
        Transaction.__init__(self)
        self.data = data
        self.date = self.today if date == None else date
        self.c_transactions = self.current_transactions()
        self.market = self.market()
        self.position = self.check_position()
        self.asset = self.check_portfolio()

    def current_transactions(self):
        return self.transactions[self.transactions.date <= (pd.to_datetime(self.date)+timedelta(0.9))]

    def market(self):
        '''
        using pivot_table to find the code with postion currently
        '''
        table = pd.pivot_table(self.c_transactions, values='quantity', index=[
                               'code', 'name'], aggfunc={'quantity': [np.sum]})
        code_name = table[table['sum'] != 0].index.tolist()
        code = [code_name[i][0] for i in range(len(code_name))]
        name = [code_name[i][1] for i in range(len(code_name))]

        market = self.data.loc[self.data.date ==
                               self.date, (['close', 'pct_chg'], code)]
        market = market.stack().reset_index()
        market['name'] = name
        market['date'] = self.date
        market.drop(columns='level_0', inplace=True)
        market.rename(columns={
                      'Symbols': 'code', 'close': 'price_current', "pct_chg": 'pct_today(%)'}, inplace=True)
        return market

    def check_position(self):
        '''
        check transaction row by row
        add one more row if no postion hold before
        edit row if some postion hold before
        delete row if no positon hold after

        return dataframe with holding number and price_cost
        '''
        position = pd.DataFrame(columns=['code', 'name'])
        num = 0
        for series in self.c_transactions.itertuples():
            if series.code not in position.code.tolist():
                position.loc[num, 'code'] = series.code
                position.loc[num, 'name'] = series.name
                position.loc[num, 'holding'] = series.quantity
                position.loc[num, 'WACP'] = series.quantity*series.price_deal
                num += 1
            else:
                index = position.code == series.code
                position.loc[index, 'holding'] += series.quantity
                position.loc[index, 'WACP'] += series.quantity * \
                    series.price_deal
                if position.loc[index, 'holding'].values == 0:
                    position.drop(index=position.index[index], inplace=True)
        try:
            position = position.merge(self.market, on=['code', 'name'])
            position['price_cost'] = position.WACP/position.holding
            position['pct_cum(%)'] = (position.price_current/position.price_cost-1)*100
            position['value'] = position.holding*position.price_current
            position['gain_daily'] = position.value * \
                position['pct_today(%)']/100/(1+position['pct_today(%)']/100)
            position['gain_cum'] = position.holding * \
                (position.price_current - position.price_cost)
        except:
            # in case of empty position
            position.loc[0, 'value'] = 0
            position.loc[0, 'gain_cum'] = 0
            position.loc[0, 'WACP'] = 1
        return position

    def check_portfolio(self):
        asset = pd.DataFrame()

        cash_outflow = self.c_transactions[[
            'turnover', 'fee', 'tax']].sum().sum()
        cash = self.init_value - cash_outflow
        mkv = self.position.value.sum()
        total_asset = mkv+cash
        asset.loc[0, 'date'] = pd.to_datetime(self.date)
        asset.loc[0, 'capital'] = self.init_value
        asset.loc[0, 'asset'] = total_asset
        asset.loc[0, 'cash'] = cash
        asset.loc[0, 'value_total'] = mkv
        asset.loc[0, 'position_rate'] = mkv/total_asset
        asset.loc[0, 'gain_total'] = total_asset-self.init_value
        asset.loc[0, 'return_total(%)'] = (total_asset/self.init_value-1)*100
        asset.loc[0, 'return_holding(%)'] = 100*self.position.gain_cum.sum() / \
            (self.position.WACP.sum())
        return asset

In [29]:
def performance():
    data=DataStored().data
    all_asset = {}
    
    for i,c_date in enumerate(map(lambda x: x.strftime("%Y%m%d"),data.date)):
        sys.stdout.write('proceeding in {}, {}/{}\r'.format(c_date,i+1,len(data.date)))
        sys.stdout.flush()
        c_asset = DailyReport(data,date=c_date).asset
        all_asset[c_date]=c_asset
    result = pd.concat(all_asset,sort=False)
    
    return result

In [30]:
def result_benchmark(result):
    asset =result[['date','asset','position_rate','return_total(%)']].copy()
    date= asset.date.dt.date.to_list()
    value = (asset.asset/200_000).round(3).to_list()
    
    rate=asset['return_total(%)'].diff().round(2).tolist()
    
    SHA = ts.pro_bar(ts_code='000001.SH', asset='I', 
               start_date=date[0].strftime("%Y%m%d"), end_date=date[-1].strftime("%Y%m%d")).close[::-1].values
    SHE = ts.pro_bar(ts_code='399001.SZ', asset='I', 
               start_date=date[0].strftime("%Y%m%d"), end_date=date[-1].strftime("%Y%m%d")).close[::-1].values
    
    SHA= (SHA/SHA[0]).round(3).tolist()[:]
    SHE= (SHE/SHE[0]).round(3).tolist()[:]
    return (date, value, rate, SHA, SHE)

In [31]:
def plot(date, value, rate, SHA, SHE):
    default_setting = {'is_symbol_show': False,
                   'is_smooth': True,
                   'is_hover_animation': False,
                   }
    line = (
        Line()
        .add_xaxis(xaxis_data=date)
        .add_yaxis(
            series_name="产品净值",
            y_axis=value,
            **default_setting
        )
        .add_yaxis(
            series_name="上证净值",
            y_axis=SHA,
            **default_setting
        )
        .add_yaxis(
            series_name="深证净值",
            y_axis=SHE,
            **default_setting
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="三都资管一期 {}".format("产品净值"),
                subtitle="From {} to {}".format(date[0], date[-1]),
                pos_left='center'
            ),
            xaxis_opts=opts.AxisOpts(type_="category"),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            legend_opts=opts.LegendOpts(
                is_show=True, pos_top='middle', pos_left="right", orient='vertical'
            ),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=0,
                    range_start=0,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=0,
                    type_="slider",
                    pos_top="92%",
                    range_start=0,
                    range_end=100,
                ),
            ],
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(245, 245, 245, 0.8)",
                border_width=1,
                border_color="#ccc",
                textstyle_opts=opts.TextStyleOpts(color="#000"),
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"),
            ),
            toolbox_opts=opts.ToolboxOpts(
                is_show=True,
                feature=opts.ToolBoxFeatureOpts(data_zoom=False, data_view=False)
            ),
        )
    )
    
    return line.render_notebook()

In [32]:
result = performance()

Loading stocks 601595.SH, proceeding 18/18
Data downloaded succesfully


In [33]:
plot(*result_benchmark(result))